In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
os.getcwd()
os.chdir('d:\\vscode_machineLearning\\internship\\sentiment-Analysis-fellowship.ai')

In [2]:
df = pd.read_csv('sentiment-analysis-dataset/IMDB_clean_data.csv',index_col=False)
df = df.drop('Unnamed: 0',axis=1) # droping the Unnamed: 0
df.dropna(inplace=True) # I had one NaN value in my dataframe.

In [3]:
import pandas as pd
import numpy as np
from pprint import pprint
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
os.getcwd()
os.chdir('d:\\vscode_machineLearning\\internship\\sentiment-Analysis-fellowship.ai')
import random

In [4]:
df = pd.read_csv('sentiment-analysis-dataset/IMDB_clean_data.csv',index_col=False)
df = df.drop('Unnamed: 0',axis=1) # droping the Unnamed: 0
df.dropna(inplace=True) # I had one NaN value in my dataframe.
df = df.reset_index()

## stemming

In [6]:
from nltk.stem import PorterStemmer
from tqdm import tqdm

In [7]:
X = df['review']
y = df['sentiment']

In [11]:
ps = PorterStemmer()

In [12]:
corpus = []
for i in tqdm(range(0,len(X)),desc ="build corpus"):
    stemmed_words = [ps.stem(word) for word in X[i].split()]
    text = ' '.join(stemmed_words)
    corpus.append(text)
    
    # print(' '.join(stemmed_words))

build corpus: 100%|██████████| 49999/49999 [01:16<00:00, 652.89it/s]


In [13]:
corpus

['one review mention watch oz episod exactli happen first thing struck oz brutal unflinch scene set right word trust show faint heart show pull punch regard sex classic use call oz nicknam given oswald maximum secur state focus mainli emerald experiment section prison cell glass front face privaci high em citi home irish death dodgi deal shadi agreement never far would say main appeal show due fact goe show forget pretti pictur paint mainstream forget forget mess first episod ever saw struck nasti say readi watch develop tast got accustom high level graphic injustic guard sold inmat kill order get away well middl class inmat turn prison bitch due lack street skill prison watch may becom comfort uncomfort get touch darker',
 'wonder littl film techniqu fashion give sometim sens realism entir actor extrem well michael sheen got voic pat truli see seamless edit guid refer diari well worth watch terrificli written perform master product one great comedi realism realli come home littl fanta

## tokenizing

In [15]:
type(corpus)

list

In [14]:
from keras.preprocessing.text import Tokenizer

## train test split

In [6]:
X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.20, random_state=42)


In [7]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (39999,)
X_test shape: (10000,)
y_train shape: (39999,)
y_test shape: (10000,)
